In [59]:
from sklearn import neighbors
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler, LabelEncoder, normalize

from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Merge, Embedding, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
np.random.seed(123)

import pickle, sys, csv
from datetime import datetime

In [2]:
df = pd.read_csv('Route15.csv', index_col=0)
df['TT_30'] = df['TT'].shift(-6)

In [3]:
cols = """['W_TT',
 'S_TT',
 'SpdAvg0',
 'occ0',
 'SpdAvg1',
 'occ1',
 'SpdAvg2',
 'occ2',
 'SpdAvg3',
 'occ3',
 'SpdAvg4',
 'occ4',
 'SpdAvg5',
 'occ5',
 'SpdTrnd0',
 'SpdTrnd1',
 'SpdTrnd2',
 'SpdTrnd3',
 'SpdTrnd4',
 'SpdTrnd5',
 'Evt0',
 'Evt1',
 'Evt2',
 'Evt3',
 'Evt4',
 'Evt5',
 'Lanes0',
 'Lanes1',
 'Lanes2',
 'Lanes3',
 'Lanes4',
 'Lanes5',
 'hldy_seq',
 'weekday',
 'timeslot',
 'TT_mean']""".replace('[', '').replace(']','').replace("'",'').replace('\n ','').split(',')

In [4]:
train_df, val_df = df[288*517:288*700], df[288*700:288*731]

In [5]:
train_df_x, train_df_y = train_df.drop('TT_30', 1), train_df[['TT_30']]
val_df_x, val_df_y = val_df.drop('TT_30', 1), val_df[['TT_30']]

In [6]:
train_df_x = train_df_x[cols]
val_df_x = val_df_x[cols]

In [7]:
cats = ['Evt0','Evt1', 'Evt2', 'Evt3', 'Evt4', 'Evt5', 'Lanes0', 'Lanes1', 'Lanes2', 'Lanes3', 'Lanes4', 'Lanes5',
 'hldy_seq', 'weekday', 'timeslot']
nums = list(set(cols) - set(cats))

In [8]:
train_df_cat_x = train_df_x[cats]
train_df_num_x = train_df_x[nums]
val_df_cat_x = val_df_x[cats]
val_df_num_x = val_df_x[nums]

In [17]:
train_df_cat_x = train_df_cat_x.fillna('-99')
val_df_cat_x = val_df_cat_x.fillna('-99')
all_df_cat_x = df[cats].fillna('-99')

In [18]:
train_df_num_x = train_df_num_x.replace([-99, '-99'], -1).fillna(-1)
val_df_num_x = val_df_num_x.replace([-99, '-99'], -1).fillna(-1)

In [19]:
for cat in cats:
    train_df_cat_x[cat] = train_df_cat_x[cat].astype('int').astype('str')
    val_df_cat_x[cat] = val_df_cat_x[cat].astype('int').astype('str')
    all_df_cat_x[cat] = all_df_cat_x[cat].astype('int').astype('str')

In [20]:
train_cat_x = train_df_cat_x.values.copy().astype('int')
val_cat_x = val_df_cat_x.values.copy().astype('int')
all_cat_x = all_df_cat_x.values.copy().astype('int')
train_num_x = train_df_num_x.values.copy().astype('int')
val_num_x = val_df_num_x.values.copy().astype('int')

In [21]:
train_y = train_df_y.values.copy().flatten()
val_y = val_df_y.values.copy().flatten()

In [22]:
les = []
for i in range(train_df_cat_x.shape[1]):
    le = LabelEncoder()
    le.fit(all_cat_x[:, i])
    les.append(le)
    train_cat_x[:, i] = le.transform(train_cat_x[:, i])
    val_cat_x[:,i] = le.transform(val_cat_x[:, i])

In [23]:
def split_features(X):
    X_list = []
    for i in range(len(X[0,:])):
        X_list.append(X[:, i])
    return X_list

In [24]:
class Model(object):
    def evaluate(self, X_val, y_val):
        assert(min(y_val) > 0)
        guessed_sales = self.guess(X_val)
        relative_err = np.absolute((y_val - guessed_sales) / y_val)
        result = np.sum(relative_err) / len(y_val)
        return result

In [66]:
class NN_with_EntityEmbedding(Model):

    def __init__(self, X_train, y_train, X_val, y_val, n_cats):
        super().__init__()
        self.data_shape = X_train.shape
        self.n_cats = n_cats
        self.unique_values = []
        for i in range(n_cats):
            self.unique_values.append(len(set(X_train[:, i])))
        self.nb_epoch = 50
        self.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
        self.max_log_y = max(np.max(np.log(y_train)), np.max(np.log(y_val)))
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)

    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list

    def __build_keras_model(self):
        models = []
        
        for i in self.unique_values:
            model = Sequential()
            model.add(Embedding(i+1, min(i+1, 10), input_length=1))
            model.add(Reshape(target_shape=(min(i+1, 10), )))
            models.append(model)
            
        print(len(models))
        for _ in range(self.n_cats, self.data_shape[1]):
            model = Sequential()
            model.add(Dense(1, input_dim=1))
            models.append(model)
            
        print(len(models))
        
        self.model = Sequential()
        self.model.add(Merge(models, mode='concat'))
        self.model.add(Dense(1000, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))        
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))        
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))        
        self.model.add(Dense(1))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('sigmoid'))

        self.model.compile(loss='mean_absolute_error', optimizer='adam')

    def _val_for_fit(self, val):
        val = np.log(val) / self.max_log_y
        return val

    def _val_for_pred(self, val):
        return np.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocessing(X_train), self._val_for_fit(y_train),
                       validation_data=(self.preprocessing(X_val), self._val_for_fit(y_val)),
                       epochs=self.nb_epoch, batch_size=128,
                       # callbacks=[self.checkpointer],
                       )
        # self.model.load_weights('best_model_weights.hdf5')
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = self.preprocessing(features)
        result = self.model.predict(features).flatten()
        return self._val_for_pred(result)

In [67]:
t1, t2 = train_cat_x[:,:12], val_cat_x[:,:12]
t1, t2 = train_cat_x[:,13:15], val_cat_x[:,13:15]
l = list(range(13)) + [13, 14]
t1, t2 = train_cat_x[:, l], val_cat_x[:, l]

In [68]:
normalizer = Normalizer()
normalizer.fit(train_num_x)

Normalizer(copy=True, norm='l2')

In [69]:
train_all = np.hstack((train_cat_x, normalizer.transform(train_num_x)))
val_all = np.hstack((val_cat_x, normalizer.transform(val_num_x)))
t1, t2 = train_all, val_all

In [70]:
m = NN_with_EntityEmbedding(t1, train_y, t2, val_y, n_cats=len(cats))

15
36


C:\Anaconda3\envs\gf\lib\site-packages\ipykernel_launcher.py:38: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 52704 samples, validate on 8928 samples
Epoch 1/50
52704/52704 [==============================] - 27s 513us/step - loss: 0.0586 - val_loss: 0.3602
Epoch 2/50
52704/52704 [==============================] - 23s 430us/step - loss: 0.0210 - val_loss: 0.0189
Epoch 3/50
52704/52704 [==============================] - 23s 433us/step - loss: 0.0171 - val_loss: 0.0138
Epoch 4/50
52704/52704 [==============================] - 23s 437us/step - loss: 0.0182 - val_loss: 0.0144
Epoch 5/50
52704/52704 [==============================] - 22s 427us/step - loss: 0.0157 - val_loss: 0.0117
Epoch 6/50
52704/52704 [==============================] - 23s 432us/step - loss: 0.0147 - val_loss: 0.0139
Epoch 7/50
52704/52704 [==============================] - 23s 432us/step - loss: 0.0147 - val_loss: 0.0183
Epoch 8/50
52704/52704 [==============================] - 23s 431us/step - loss: 0.0147 - val_loss: 0.0100
Epoch 9/50
52704/52704 [==============================] - 23s 431us/step - loss: 0.0143 - val_l

In [31]:
m.guess(t1)

array([ 144.44680786,  145.77832031,  145.94734192, ...,  141.63539124,
        141.95899963,  142.79364014], dtype=float32)